In [3]:
import os
import numpy as np
import plotly.graph_objects as go 
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime, timezone
import json  
import pandas as pd

### Извлечение данных

In [ ]:

# Check all file names in data directory
dir_path = "C:\\Users\\picni\\OneDrive\\Рабочий стол\\Работа\\opto_ident\\Данные\\204\\Data"
files_indir_names = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files_indir_names.append(dir_path + "\\" + path)
data = {}
# Iterate all files in data directory and add <data> information 
for file_name in tqdm(files_indir_names):
    with open(file_name, 'r') as f:
        file = f.read() 
    soup = BeautifulSoup(file, 'xml')
    names = soup.find_all('data')
    res = []
    for name in names:
        name = name.text.split(",")
        res.append([float(i) for i in name])
    res = np.array(res)

    # time exatraction 
    start_time = soup.find_all('startDateTimeIndex')[0].text
    #time to unix epoch time  
    utc_time = datetime.strptime(start_time[0:-6],
                             "%Y-%m-%dT%H:%M:%S.%f")
    data[file_name] = [res,  utc_time.timestamp()]


In [ ]:
names = ["LAF","ST","AST","TMP"]
fig = go.Figure()
for num in range(4):
    fig.add_trace(go.Scatter(
                        y=data["channel 2_20230912104409796.xml"][0][:,num],
                        line=dict(width=2),
                        name = names[num]              
    ))
fig.update_layout(
    title=dict(text="data"),
    xaxis_title="step", yaxis_title="value"
)
fig.show()


### Запускай все что ниже для 3D и карты высот

### Построение зависимости глубины температуры и времени

In [ ]:
# Check all file names in data directory
dir_path = "C:\\Users\\picni\\OneDrive\\Рабочий стол\\Работа\\opto_ident\\Данные\\204\\Data"
files_indir_names = []
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files_indir_names.append(dir_path + "\\" + path)
temp = []
time = []
laf = []
# Iterate all files in data directory and add <data> information 
for file_name in tqdm(files_indir_names):
    with open(file_name, 'r') as f:
        file = f.read() 
    soup = BeautifulSoup(file, 'xml')
    names = soup.find_all('data')
    res = []
    for name in names:
        name = name.text.split(",")
        res.append([float(i) for i in name])
    res = np.array(res)
    temp.append(res[:,3])
    # time exatraction 
    start_time = soup.find_all('startDateTimeIndex')[0].text
    #time to unix epoch time  
    utc_time = datetime.strptime(start_time[0:-6],
                             "%Y-%m-%dT%H:%M:%S.%f")
    time.append(utc_time.timestamp())
    laf.append(res[:,0])



### Для сохранения в JSON

In [ ]:
json_dict = dict()
indx = 0
for i in time:
    json_dict[i] = {"temp": temp[indx].tolist(), "laf": laf[indx].tolist()}
    indx += 1
json_object = json.dumps(json_dict, indent = 4)
with open("data.json", "w") as outfile:
    outfile.write(json_object)

### Для считывания из JSON

In [2]:
with open('data.json', 'r') as openfile:
    json_object = json.load(openfile)
time = list(json_object.keys())
time = np.array([float(i) for i in time])
laf = np.array([i['laf'] for i in json_object.values()])
temp = np.array([i['temp'] for i in json_object.values()])

# Прорежаем данные 
DELIM = 1
temp_new = temp[:,::DELIM]
laf_new = laf[0][::DELIM]


In [17]:
# covert time to datetime format
df = pd.DataFrame()
df['time'] = time
df['time'] = pd.to_datetime(df['time'] * 10**9)

### Карта высот

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(
    z=temp_new,
    zmax=70,  
    zmin=20,
    x = laf_new,
    y = df['time'],
))
fig.update_layout(
    xaxis_title = "Depth, m",
    yaxis_title = "Date",
)
fig.write_html("heatmap.html")
fig.show()

### Функция расчета корреляции

In [18]:
def corr(x, y):
    x = (x - x.mean())
    x /= np.linalg.norm(x)
    y = (y - y.mean())
    y /= np.linalg.norm(y)
    result = np.correlate(x, y, mode='full')
    result = result[result.size//2:]
    index_max = np.argmax(result)
    return index_max

In [ ]:
offset = {}
ref_idx = 13450
for i in range(250):
    off_idx =  ref_idx - i
    x = temp[340:520, ref_idx]
    y = temp[340:520, off_idx]
    result = corr(y,x)
    offset[laf_new[off_idx]] = result
fig = go.Figure()
fig.add_trace(go.Scatter(
    x =list(offset.keys()),
    y = np.array(list(offset.values()))*12*60,
))
fig.write_html("file.html")

fig.show()

In [ ]:
fig = go.Figure()
for i in range(len(time)):
    temp_new = temp[i,:]
    fig.add_trace(go.Scatter(
        y = temp_new,
        name  = str(df['time'][i]),
        x = laf_new
    ))
fig.update_layout(
    xaxis_title = "Depth, m",
    yaxis_title = "Temp, C",
)
fig.write_html("file.html")

fig.show()

In [171]:
fig = go.Figure()

# раз в сколько берем температуру, 
delim = 10
len_ = int(len(temp[0,:]) / delim)

max_depth = 1618
min_depth = 1585

for i in range(len_):
    if (laf_new[i*delim] > min_depth and laf_new[i*delim] < max_depth):
        temp_new = temp[:,i*delim]
        fig.add_trace(go.Scatter(
            y = temp_new,
            x = df['time'],
            name = laf_new[i*delim]
        ))
fig.update_layout(
    xaxis_title = "Date",
    yaxis_title = "Temp, C",
)
fig.write_html("temp(time) same depth.html")
fig.show()

In [225]:
df_sort = pd.DataFrame()
df_sort['laf'] = laf_new

# Начальная и конечная высота для сравнения
start_laf = 1585
end_laf = 1600.0
# начальное и конечное время
start_time = '2023-09-08 13:00:00'
end_time = '2023-09-09 04:40:00'

dt = 12 * 60
start_idx = df_sort.iloc[(df_sort['laf']-start_laf).abs().argsort()[:1]].index.values[0]
end_idx = df_sort.iloc[(df_sort['laf']-end_laf).abs().argsort()[:1]].index.values[0]

start_idx_time = df.iloc[(df['time']-pd.to_datetime(start_time)).abs().argsort()[:1]].index.values[0]
end_idx_time = df.iloc[(df['time']-pd.to_datetime(end_time)).abs().argsort()[:1]].index.values[0]

delay = []
for i in range(start_idx+1, end_idx, 1):
    offset = corr(temp[start_idx_time:end_idx_time, start_idx], temp[start_idx_time:end_idx_time, i])
    delay.append(offset)
delay = np.array(delay)
# delay = np.diff(np.array(delay))

In [226]:
(end_laf - start_laf) / (dt * 6)

0.003472222222222222

In [228]:
from scipy.optimize import curve_fit

x = np.arange(31,len(delay))
def func(x, a, b):
    return a  * x + b
cfs, _ = curve_fit(func, x, delay[31:])

In [229]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y = delay
))
fig.add_trace(go.Scatter(
    y = cfs[0]*x + cfs[1],
    x = x
))
fig.show()

In [232]:
y = cfs[0]*x + cfs[1]

(end_idx - start_idx) / y[-1] / dt 

0.04125478177964247

In [231]:
dT = np.diff(cfs[0]*x + cfs[1])

In [196]:
d_laf = laf_new[1]-laf_new[0]
sum = 0
for i in range(len(delay)-1):
    sum += (d_laf / (cfs[0] * 12 * 60))

In [197]:
sum

0.06390592572987738